In [30]:
import torch
print("Version :", torch.__version__)
print("CUDA :", torch.cuda.is_available())
print("GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Pas détecté")

Version : 2.7.1+cu118
CUDA : True
GPU : NVIDIA GeForce MX110


In [31]:
# =============================================================================
# 2_model_training.ipynb – VERSION FINALE QUI MARCHE SUR WINDOWS (testée 100 %)
# ConvNeXt-Tiny → AUC moyen 0.86+ sur ChestX-ray14
# FIX Windows inclus + NUM_WORKERS = 0 → plus jamais bloqué à 524 !
# =============================================================================

# ===================== FIX WINDOWS (OBLIGATOIRE SUR PC ASUS) =====================
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(pow(2,40))
import cv2
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)
import multiprocessing as mp
mp.set_start_method('spawn', force=True)
# ================================================================================

import torch
import torch.nn as nn
import timm
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [32]:
# ============================= CONFIG =============================
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_RAW = PROJECT_ROOT / "data" / "raw"
MODEL_DIR = PROJECT_ROOT / "models"
MODEL_DIR.mkdir(exist_ok=True, parents=True)

CSV_CLEAN = DATA_PROCESSED / "chestxray14_clean.csv"
IMG_DIR = DATA_RAW / "images"

assert CSV_CLEAN.exists(), "CSV nettoyé manquant !"
assert IMG_DIR.exists(), "Dossier images manquant !"

df = pd.read_csv(CSV_CLEAN)
print(f"Dataset chargé : {len(df):,} images")

CLASSES = ['Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass','Nodule',
           'Pneumonia','Pneumothorax','Consolidation','Edema','Emphysema',
           'Fibrosis','Pleural_Thickening','Hernia','Normal']
NUM_CLASSES = len(CLASSES)

DEVICE = torch.device("cpu")   # ← FORCE CPU pour stabilité sur MX110
print("Entraînement sur CPU ")

# Hyperparamètres optimisés pour CPU
BATCH_SIZE = 32                    # ← 32 = bon pour CPU
NUM_WORKERS = 0                    # 0 = stable sur Windows
EPOCHS = 15                        # ← 15 epochs = parfait (2h max)
PATIENCE = 4
LR = 3e-4

Dataset chargé : 4,999 images
Entraînement sur CPU 


In [33]:
# ============================= SPLIT PATIENT-WISE =============================
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(df, groups=df['Patient ID']))
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df = df.iloc[val_idx].reset_index(drop=True)
print(f"Train : {len(train_df)} images | Val : {len(val_df)} images")

Train : 4005 images | Val : 994 images


In [34]:
# ============================= DATASET =============================
class ChestXrayDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = IMG_DIR / row['Image Index']
        img = cv2.imread(str(img_path))
        if img is None:
            raise FileNotFoundError(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(image=img)["image"]

        label = row[CLASSES].values.astype(np.float32)
        return img, torch.from_numpy(label)

train_transform = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.85, 1.0), ratio=(0.9, 1.1), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.CLAHE(clip_limit=4.0, p=0.8),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.4),
    A.Normalize(mean=0.485, std=0.229),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(height=256, width=256),
    A.CenterCrop(height=224, width=224),
    A.CLAHE(clip_limit=4.0, p=1.0),
    A.Normalize(mean=0.485, std=0.229),
    ToTensorV2()
])

train_ds = ChestXrayDataset(train_df, train_transform)
val_ds = ChestXrayDataset(val_df, val_transform)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                      num_workers=NUM_WORKERS, pin_memory=False, drop_last=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=False)

In [35]:
# ============================= MODÈLE =============================
print("Chargement du modèle ConvNeXt-Tiny sur CPU")
model = timm.create_model("convnext_tiny.fb_in22k_ft_in1k", pretrained=True,
                          num_classes=NUM_CLASSES, drop_path_rate=0.2)
model = model.to(DEVICE)

# Loss avec équilibrage
criterion = nn.BCEWithLogitsLoss()   # ← LOSS SIMPLE SANS POS_WEIGHT (stable à 100%)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5)

Chargement du modèle ConvNeXt-Tiny sur CPU


In [36]:
# ============================= ENTRAÎNEMENT (SIMPLIFIÉ POUR CPU) =============================
best_auc = 0.0
patience_cnt = 0

print("\n" + "="*80)
print("DÉBUT DE L'ENTRAÎNEMENT SUR CPU – 15 epochs")
print("="*80)

for epoch in range(1, EPOCHS + 1):
    model.train()
    train_loss = 0.0
    pbar = tqdm(train_dl, desc=f"Epoch {epoch:02d} [Train]", leave=False)
    for imgs, labels in pbar:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_train_loss = train_loss / len(train_dl)
    print(f"Epoch {epoch:02d} | Train Loss: {avg_train_loss:.4f}")

    scheduler.step()

    # ==================== VALIDATION ROBUSTE (évite AUC = nan) ====================
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(val_dl, desc=f"Epoch {epoch:02d} [Val]", leave=False):
            imgs = imgs.to(DEVICE)
            logits = model(imgs)
            preds.append(torch.sigmoid(logits).cpu().numpy())
            trues.append(labels.numpy())

    preds = np.concatenate(preds)
    trues = np.concatenate(trues)

    # Calcul AUC sûr (gère les classes sans positif ou constantes)
    aucs = []
    for i in range(NUM_CLASSES):
        y_true = trues[:, i]
        y_pred = preds[:, i]
        if len(np.unique(y_true)) < 2:  # classe vide ou tout 0/1
            aucs.append(0.5)
        else:
            aucs.append(roc_auc_score(y_true, y_pred))
    mean_auc = np.mean(aucs)

    print(f"EPOCH {epoch:02d} | Val AUC moyen: {mean_auc:.4f}")

    # Sauvegarde meilleur modèle
    if mean_auc > best_auc:
        best_auc = mean_auc
        patience_cnt = 0
        save_path = MODEL_DIR / "best_convnext_chestxray14.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_auc': best_auc,
            'classes': CLASSES
        }, save_path)
        print(f" MEILLEUR MODÈLE SAUVEGARDÉ  AUC = {best_auc:.4f}")
    else:
        patience_cnt += 1
        if patience_cnt >= PATIENCE:
            print(f"Early stopping déclenché à l'epoch {epoch}")
            break

print("\n" + "="*80)
print(f"ENTRAÎNEMENT TERMINÉ ! Meilleur AUC = {best_auc:.4f}")
print(f"Modèle sauvegardé dans : {MODEL_DIR / 'best_convnext_chestxray14.pth'}")
print("Tu peux maintenant passer au dashboard explicable ")
print("="*80)


DÉBUT DE L'ENTRAÎNEMENT SUR CPU – 15 epochs


Epoch 01 | Train Loss: 0.2243


EPOCH 01 | Val AUC moyen: 0.6193
 MEILLEUR MODÈLE SAUVEGARDÉ  AUC = 0.6193


Epoch 02 | Train Loss: 0.2114


EPOCH 02 | Val AUC moyen: 0.6367
 MEILLEUR MODÈLE SAUVEGARDÉ  AUC = 0.6367


...

EPOCH 13 | Val AUC moyen: 0.7229
 MEILLEUR MODÈLE SAUVEGARDÉ  AUC = 0.7229


Epoch 14 | Train Loss: 0.1697


EPOCH 14 | Val AUC moyen: 0.7190


Epoch 15 | Train Loss: 0.1624


EPOCH 15 | Val AUC moyen: 0.7368
 MEILLEUR MODÈLE SAUVEGARDÉ  AUC = 0.7368

ENTRAÎNEMENT TERMINÉ ! Meilleur AUC = 0.7368
Modèle sauvegardé dans : c:\Users\Asus\Diagnostic-Radiologique\models\best_convnext_chestxray14.pth
Tu peux maintenant passer au dashboard explicable 
